Planificación usando Pyomo - Parte 2
===

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia

---

Haga click [aquí](https://github.com/jdvelasq/workshop-asocio-2018/tree/master/) para acceder al repositorio online.

Haga click [aquí](http://nbviewer.jupyter.org/github/jdvelasq/workshop-asocio-2018/tree/master/) para explorar el repositorio usando `nbviewer`. 

### Definición del problema

 
 
* **Planificacion:**    4 etapas. 

* **Plantas hidraulicas:**



| Planta | Vol Max | Factor | Vol Inicial |  
|:------:|:-------:|:------:|:-----------:|
|   1    | 100     |  1.0   |    75       |
|   2    | 100     |  1.0   |    75       |

* **Aportes:**

| Planta | Etapa | Aportes |
|:------:|:-----:|:-------:|
|    1   |   1   |   21    |
|    1   |   2   |   15    |
|    1   |   3   |   12    |
|    1   |   4   |   18    |
|    2   |   1   |   14    |
|    2   |   2   |    8    |
|    2   |   3   |    5    |
|    2   |   4   |   11    |



* **Plantas térmicas:**

| Planta | Gen máxima | Costo Combustible |
|:------:|:----------:|:-----------------:|
|   1    |    40      |      15           | 
|   2    |    45      |      15           |  
    
    
 
 
* **Racionamiento:**   
    * Costo racionamiento (CR) = 1000 para todas las etapas. 
 
 
* **Demanda:**  50 para todas las etapas 
 

### Inicialización de Pyomo

In [1]:
%%writefile model.py

from pyomo.environ import *

model = ConcreteModel()


Writing model.py


### Datos de entrada del modelo

In [2]:
%%writefile -a model.py

## Definicion de variables en Pyomo

## Etapas
P  = range(4)   # 4 etapas
HD = range(2)   # 2 plantas hidro
TM = range(2)   # 2 plantas termicas


## etapas:
##       1     2     3     4       ## variable
## ----------------------------------------------------------

## costos de racionamiento 
CR = [ 1000.,   # etapa 1 
       1000.,   # etapa 2
       1000.,   # etapa 3
       1000.]   # etapa 4

## demanda de energia
D  = [ 50.,  # etapa 1  
       50.,  # etapa 2  
       50.,  # etapa 3 
       50.]  # etapa 4    

## costo de combustibles
CC = [ 15,  # planta termica 1  
       15]  # planta termica 2 

## generación termica maxima
G = [  45,  # planta termica 1
       40]  # planta termica 2  

## volumen inicial
Vo = [ 75,  # planta hidro 1
       75]  # planta hidro 2      

## caudal máximo turbinado
Q = [ 50,   # planta hidro 1
      45]   # planta hidro 2


## Aporter hidrologicos         
A = {(1, 1):21, # (planta hidro 1, etapa) : aporte
     (1, 2):15, #
     (1, 3):12, #
     (1, 4):18, #
     (2, 1):14, # (planta hidro 2, etapa) : aporte
     (2, 2):8,  #
     (2, 3):5,  #
     (2, 4):11} # 

Appending to model.py


### Creacion de las variables

In [3]:
%%writefile -a model.py

## energia[etapas]
model.R = Var(P, within=NonNegativeReals)                              

## generacion_termica[etapa, planta_termica]
model.G = Var(P, TM, within=NonNegativeReals)                             

## caudal_turbinado[etapa, planta_hidro]
model.Q = Var(P, HD, within=NonNegativeReals)

## volumen_final[etapa, planta_hidro]
model.V = Var(P, HD, within=NonNegativeReals)

# vertimientos[etapa, planta_hidro]
model.S = Var(P, HD, within=NonNegativeReals)


Appending to model.py


### Funcion objetivo

La funcion objetivo busca minimizar el costo total del racionamiento mas el costo total de la generacion termica:


$$\text{minimize} ~ \sum_{p=1}^{P} ~\Bigg\{ CR * R_p + \sum_{t=1}^T CC_t * GT_{p,t} ~\Bigg\}$$







In [4]:
%%writefile -a model.py

## Definicion de la funcion objetivo
#                                 costo              costo generacion
#                                 racionamiento      termica 
model.obj = Objective(expr = sum( CR[i]*model.R[i] + sum( CC[j]*model.G[i, j] for j in TM) for i in P ))


Appending to model.py


### Demanda

Por cada periodo de tiempo $p$ debe cumplirse que:

$$ R_p + \Bigg\{ \sum_{h=1}^H \rho_h * Q_{hp} \Bigg\} + \Bigg\{ \sum_{t=1}^T GT_{t p} \Bigg\} = dem_p$$


In [5]:
%%writefile -a model.py

## una restriccion por cada etapa
def demanda_rule(model, p):
        #      racionamiento  gen. termica                    gen hidro
        return model.R[p]  +  sum(model.G[p,j] for j in TM) + sum(model.Q[p,j] for j in HD)== D[p]

## crea una regla por cada etapa    
model.demanda = Constraint(P, rule=demanda_rule)



Appending to model.py


### Continuidad de los volumenes de los embalses

Para el primer periodo, el embalse inicial $Vol_{h,0}$ es conocido, entonces:

$$Vol_{h,1} +Q_{h,1}+Ver_{h,1}=A_{h,1}+ Vol_{h,0}$$

In [6]:
%%writefile -a model.py

# una restriccion por cada etapa, por cada planta hidro
def continuidad_rule(model, p, hd):
    if P[p] == 0 :
        #      volumen         caudal          vertimientos     volumen inicial
        #      final           turbinado                        aportes
        return model.V[p,hd] + model.Q[p,hd] + model.S[p,hd] == Vo[hd]+ A[hd+1,1]     
    else:
        #       volumen           volumen         caudal          vertimmientos     aportes
        #       inicial           final           turbinado               
        return -model.V[p-1,hd] + model.V[p,hd] + model.Q[p,hd] + model.S[p,hd] ==  A[hd+1,p+1]    

model.continuidad = Constraint(P, HD, rule=continuidad_rule)


Appending to model.py


### Caudal turbinado máximo

In [7]:
%%writefile -a model.py

## una regla por cada etapa y cada planta hidro
def qmax_rule(model, p, HD):
    return model.Q[p, HD] <= Q[HD]

model.qmax = Constraint(P, HD, rule = qmax_rule)


Appending to model.py


### Generacion térmica máxima

In [8]:
%%writefile -a model.py

def gtmax_rule(model, p, TM):
    return model.G[p, TM] <= G[TM]

model.gtmax = Constraint(P, TM, rule = gtmax_rule)

Appending to model.py


### Solución

In [9]:
!pyomo solve --solver=glpk model.py

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.01] Creating model
[    0.01] Applying solver
[    0.02] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: feasible
      Function Value: 990.0
    Solver results file: results.yml
[    0.02] Applying Pyomo postprocessing actions
[    0.02] Pyomo Finished


In [10]:
!cat results.yml

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 990.0
  Upper bound: 990.0
  Number of objectives: 1
  Number of constraints: 29
  Number of variables: 37
  Number of nonzeros: 67
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.008816957473754883
# ----------------------------------------------------------
#   Solution Information
# ---------------------------------

In [11]:
!rm model.py results.yml

Planificación usando Pyomo - Parte 2
===

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia

---

Haga click [aquí](https://github.com/jdvelasq/workshop-asocio-2018/tree/master/) para acceder al repositorio online.

Haga click [aquí](http://nbviewer.jupyter.org/github/jdvelasq/workshop-asocio-2018/tree/master/) para explorar el repositorio usando `nbviewer`. 